## Utils

In [11]:
## Utils
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
import torch
import random
import numpy as np

def set_pytorch_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    torch.use_deterministic_algorithms(True)

set_pytorch_seed(0)
from tsl.data.preprocessing.scalers import MinMaxScaler
from models.utils.MPNN import MPNN
from models.baseline.MPNN_ODE import MPNN_ODE
import torch
import sympytorch
from torch_geometric.data import Data
import sympy as sp
import copy
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from post_processing import get_model, make_callable, plot_predictions
from torch.optim import LBFGS


def get_scaler(data, tr_perc = 0.8, scale_range = (-1, 1)):
    raw_data = data.raw_data_sampled.detach().cpu().numpy() # shape (IC, T, N, 1)
    tr_len = raw_data.shape[1]
    raw_data = raw_data[0, :int(tr_perc*tr_len), :, :]
    scaler = MinMaxScaler(out_range=scale_range)
    scaler.fit(raw_data.flatten())
    
    return scaler


  
def eval_real_epid_int(data, countries_dict, build_symb_model, inferred_coeffs, scaler=None, use_euler=False, tr_perc = 0.8):
    y_true = data[0].y.detach().cpu().numpy()
    y_pred = np.zeros_like(y_true)
    
    
    for country_name, node_idx in countries_dict.items():
        if country_name == "Germany":
            continue
        symb_model = build_symb_model(country_name, inferred_coeffs)
        
        # print(f"{country_name}")
        data_0 = data[0]
        if scaler is not None:
            tmp = scaler.transform(data[0].x)
            data_0 = data[0]
            data_0.x = tmp
        
        if use_euler:
            symb_model.integration_method = "euler"
            data_0.t_span = torch.arange(y_true.shape[0] + 1, device=data_0.x.device, dtype=data_0.t_span.dtype)
        
        try:
            pred = symb_model(data_0).detach().cpu().numpy()
        except AssertionError:
            print("Failed")
            continue
        
        if scaler is not None:
            pred = scaler.inverse_transform(pred)
        
        y_pred[:, node_idx, :] = pred[:, node_idx, :]
    
    tr_len = y_true.shape[0]
    tr_end = int(tr_perc * tr_len)
    y_true_val = y_true[tr_end:, :, :]
    y_pred_val = y_pred[tr_end:, :, :] 
    
    return y_true, y_pred, y_true_val, y_pred_val 


def eval_real_epid_journal(data, countries_dict, build_symb_model, inferred_coeffs, tr_perc = 0.8, step_size=1.0, scaler = None):
    def get_dxdt_pred(data, symb_model):
        dxdt_pred = []
        for snapshot in data:
            dxdt_pred.append(symb_model(snapshot))
        
        return torch.stack(dxdt_pred, dim=0)
    
    def sum_over_dxdt(dxdt_pred):
        out = []
        for i in range(dxdt_pred.shape[0]):
            out.append(torch.sum(step_size*dxdt_pred[0:i+1, :, :], dim=0)) 
        
        return torch.stack(out, dim=0)
        
    def integrate(out, x0):
        pred = []
        for i in range(1, out.shape[0]):
            pred.append(x0 + out[i, :, :])
        return torch.stack(pred, dim=0)
      
    x0 = data[0].x
    y_true = torch.stack([d.x for d in data[1:]], dim=0).detach().cpu().numpy()
    y_pred = np.zeros_like(y_true)
     
    for country_name, node_idx in countries_dict.items():
        symb_model = build_symb_model(country_name, inferred_coeffs)
        symb_model.predict_deriv = True
        dxdt_pred = get_dxdt_pred(data, symb_model)
        out = sum_over_dxdt(dxdt_pred)
        pred = integrate(out, x0).detach().cpu().numpy()
        y_pred[:, node_idx, :] = pred[:, node_idx, :]
    
    if scaler is not None:
        y_pred = scaler.inverse_transform(y_pred)
        y_true = scaler.inverse_transform(y_true)
    
    tr_len = y_true.shape[0]
    tr_end = int(tr_perc * tr_len)
    y_true_val = y_true[tr_end:, :, :]
    y_pred_val = y_pred[tr_end:, :, :] 
    
    return y_true, y_pred, y_true_val, y_pred_val 
        


import pandas as pd
# from torch.optim import Adam
from torch_geometric.loader import DataLoader
# from scipy.optimize import minimize
from typing import Dict
# import optuna

    
def fit_param_per_country_gd(data, countries_dict: Dict[str, int], model_path, build_symb_model, epochs=100, loss_fn=torch.nn.L1Loss(), device='cuda:0', lr=1e-3, 
                             patience=10, log=10, save_file = "inferred_coeff.csv", scaler = None, tr_perc = 0.8):
    
    def get_predictions(data_0, model, node_idx):
        try:
            y_pred = model(data_0)
        except AssertionError:
            y_pred = torch.zeros_like(data[0].y, device=data[0].y.device)
        
        return torch.nan_to_num(y_pred[:, node_idx, :], nan=-1.0)

 
    
    N = data[0].x.shape[1]
    T = data[0].y.shape[0]
    tr_end = int(T*tr_perc)
    
    node_models = [build_symb_model().to(device) for _ in range(N)]

    for model in node_models:
        for param in model.parameters():
            if param.data > 1e-5:
                param.requires_grad_ = True
            else:
                param.requires_grad_ = False

    optimizers = [
        LBFGS(model.parameters(), lr=lr, line_search_fn="strong_wolfe",
              tolerance_grad=1e-32, tolerance_change=1e-32)
        for model in node_models
    ]

    results_df = pd.DataFrame()

    for country_name, node in countries_dict.items():
        best_val_loss = float('inf')
        best_epoch = 0
        best_model_state = None

        model = node_models[node]
        optimizer = optimizers[node]

        print(f"\nProcessing country {country_name}")
        for epoch in range(epochs):
            train_node_loss = 0.
            model.train()
            data_0 = data[0]
            y_true, y_pred = [], []
            if scaler is not None:
                tmp = scaler.transform(data[0].x)
                data_0 = data[0]
                data_0.x = tmp
                data_0 = data_0.to(device)
            
            def closure():
                nonlocal y_true, y_pred
                optimizer.zero_grad()
                y_pred = get_predictions(data_0, model, node)
                y_true = data[0].y[:, node, :].to(device)
                if scaler is not None:
                    y_true = scaler.transform(y_true.cpu()).to(device)
                    
                loss = loss_fn(y_pred[:tr_end], y_true[:tr_end])
                loss.backward()
                return loss

            loss = optimizer.step(closure)
            train_node_loss += loss.item()

            
            with torch.no_grad():
                model.eval()
                y_pred_val = get_predictions(data_0, model, node)[tr_end:]
                val_loss = loss_fn(y_pred_val, y_true[tr_end:])

            if val_loss < best_val_loss:
                best_epoch = epoch
                best_val_loss = val_loss
                best_model_state = copy.deepcopy(model.state_dict())
            elif epoch - best_epoch > patience:
                print(f"Early stopping at epoch {epoch}")
                break

            if epoch % log == 0:
                print(f"Epoch {epoch}, train Loss: {train_node_loss:.4f}, valid loss: {val_loss:.4f}")

        model.load_state_dict(best_model_state)
        h_net = model.conv.model.h_net
        g_net = model.conv.model.g_net
        self_int_coeffs = torch.cat([p.detach().cpu().flatten() for p in h_net.parameters()]).numpy()
        pairwise_int_coeffs = torch.cat([p.detach().cpu().flatten() for p in g_net.parameters()]).numpy()
        coeffs = np.concatenate([self_int_coeffs, pairwise_int_coeffs])
        results_df[country_name] = coeffs
        print(f"Inferred coeffs for {country_name}: {coeffs}")

    results_df.to_csv(f"{model_path}/{save_file}")
    

def fit_param_per_country_pd(data_train, data_valid, countries_dict: Dict[str, int], model_path, build_symb_model, epochs=100, loss_fn=torch.nn.L1Loss(), device='cuda:0', lr=1e-3,
                              batch_size=64, patience=10, log=10, save_file = "inferred_coeff.csv"):
    
    def get_predictions(batch_data, model, node_idx):
        y_pred = model(batch_data)
        y_true = batch_data.y

        y_pred = torch.reshape(y_pred, (batch_data.num_graphs, N, -1))
        y_true = torch.reshape(y_true, (batch_data.num_graphs, N, -1))

        return y_true[:, node_idx, :], y_pred[:, node_idx, :]

    def eval_model(model, valid_loader, node_idx, loss_fn):
        model.eval()
        y_pred = []
        y_true = []
        with torch.no_grad():
            for batch_valid in valid_loader:
                y_true_b, y_pred_b = get_predictions(
                    batch_data=batch_valid,
                    model=model,
                    node_idx=node_idx
                )
                y_true.append(y_true_b)
                y_pred.append(y_pred_b)

            y_pred = torch.cat(y_pred, dim=0)
            y_true = torch.cat(y_true, dim=0)

            valid_loss = loss_fn(y_pred, y_true)

        return valid_loss.item()

    N, _ = data_train[0].x.shape
    node_models = [build_symb_model().to(device) for _ in range(N)]

    for model in node_models:
        for param in model.parameters():
            param.requires_grad_ = True

    optimizers = [
        LBFGS(model.parameters(), lr=lr, line_search_fn="strong_wolfe",
              tolerance_grad=1e-32, tolerance_change=1e-32)
        for model in node_models
    ]

    results_df = pd.DataFrame()

    for country_name, node in countries_dict.items():
        train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
        valid_loader = DataLoader(data_valid, batch_size=len(data_valid), shuffle=False)
        best_val_loss = float('inf')
        best_epoch = 0
        best_model_state = None

        model = node_models[node]
        optimizer = optimizers[node]

        print(f"\nProcessing country {country_name}")
        for epoch in range(epochs):
            train_node_loss = 0.
            count = 0
            model.train()

            for batch_data in train_loader:
                batch_data = batch_data.to(device)
                count += 1

                def closure():
                    optimizer.zero_grad()
                    y_true, y_pred = get_predictions(batch_data, model, node)
                    loss = loss_fn(y_pred, y_true)
                    loss.backward()
                    return loss

                loss = optimizer.step(closure)
                train_node_loss += loss.item()

            val_loss = eval_model(
                model=model,
                valid_loader=valid_loader,
                node_idx=node,
                loss_fn=loss_fn
            )

            if val_loss < best_val_loss:
                best_epoch = epoch
                best_val_loss = val_loss
                best_model_state = copy.deepcopy(model.state_dict())
            elif epoch - best_epoch > patience:
                print(f"Early stopping at epoch {epoch}")
                break

            if epoch % log == 0:
                print(f"Epoch {epoch}, train Loss: {train_node_loss / count:.4f}, valid loss: {val_loss:.4f}")

        model.load_state_dict(best_model_state)
        h_net = model.conv.model.h_net
        g_net = model.conv.model.g_net
        self_int_coeffs = torch.cat([p.detach().cpu().flatten() for p in h_net.parameters()]).numpy()
        pairwise_int_coeffs = torch.cat([p.detach().cpu().flatten() for p in g_net.parameters()]).numpy()
        coeffs = np.concatenate([self_int_coeffs, pairwise_int_coeffs])
        results_df[country_name] = coeffs
        print(f"Inferred coeffs for {country_name}: {coeffs}")

    results_df.to_csv(f"{model_path}/{save_file}")
        


# def fit_param_per_country(data, countries_dict, model_path, build_symb_model,init_params,  tr_perc = 0.8, scaler=None,
#                           use_euler=False, min_method = "BFGS"):
#     tr_len = data.raw_data_sampled.shape[1]
#     def optim_fun(params):
#         g_symb, h_symb = build_symb_model(params)
#         symb_model = get_model(
#             g = g_symb,
#             h = h_symb,
#             message_passing=False,
#             include_time=False,
#             integration_method='rk4'
#         )
        
#         data_0 = data[0]
#         if scaler is not None:
#             tmp = scaler.transform(data[0].x)
#             data_0 = data[0]
#             data_0.x = tmp
            
#         if use_euler:
#             symb_model.integration_method = "euler"
#             data_0.t_span = torch.arange(tr_len, device=data_0.x.device, dtype=data_0.t_span.dtype)
            
#         out = symb_model(data_0)
        
#         y_pred = out[:int(tr_len*tr_perc), node_idx, 0].detach().cpu().numpy()
#         y_true = data[0].y[:int(tr_len*tr_perc), node_idx, 0].detach().cpu().numpy()
        
#         if scaler is not None:
#             y_pred = scaler.inverse_transform(y_pred)
#         loss = mean_squared_error(y_true.flatten(), y_pred.flatten())
#         return loss
        
#     results_df = pd.DataFrame()
    
#     # init_params=[-0.0039747115, 2.4683, 2.464987]
#     # Optimization
#     for country_name, node_idx in countries_dict.items():
#         print(f"Processing {country_name}")
#         result = minimize(optim_fun, init_params, method=min_method)
#         results_df[country_name] = result.x
#         print(result.x)
    
#     results_df.to_csv(f"{model_path}/inferred_coeff.csv")     


# def fit_coeffs_per_country(data, countries_dict, model_path, symb_model:MPNN_ODE):
    
#     assert not symb_model.conv.model.message_passing, "This function works only for models of type H(x_i) + sum(G(x_i, x_j))"
    
#     T = len(data)
#     N = data[0].x.shape[0]
    
#     collate_fn = lambda samples_list: samples_list
#     train_loader = DataLoader(data, batch_size=T, shuffle=False, collate_fn=collate_fn)
#     all_data = next(iter(train_loader))
    
#     Y = torch.stack(
#         [d.y for d in data],
#         dim=0
#     )   # shape (T, N, 1)
    
#     # forward pass
#     _ = symb_model(all_data)
#     self_int_out = symb_model.conv.model.upduate_out
#     pairwise_int_out = symb_model.conv.model.message_out
    
#     self_int_out = torch.reshape(self_int_out, (T, N, -1)) 
#     pairwise_int_out = torch.reshape(pairwise_int_out, (T, N, -1))
    
#     def optim_fun(params):
#         a, b, = params
        
#         X_in = torch.stack([
#             self_int_out[:, node_idx, 0],
#             pairwise_int_out[:, node_idx, 0]
#         ], dim = 1)
        
#         y_true = Y[:, node_idx, 0].detach().cpu().numpy()
#         y_pred = (a*X_in[:, 0] + b*X_in[:, 1]).detach().cpu().numpy()
#         loss = mean_squared_error(y_true, y_pred)
#         return loss
        
    
#     results_df = pd.DataFrame()
#     for country_name, node_idx in countries_dict.items():
#         print(f"Processing {country_name}")
#         result = minimize(optim_fun, [1., 1.], method='L-BFGS-B')        
#         params = result.x  
#         print(params)
#         results_df[country_name] = params
        
#     results_df.to_csv(f"{model_path}/inferred_coeff.csv")
        
    
    
# def euler_int(symb_model, data_0, T):
#     pred = []
#     x = data_0.x[-1]
#     edge_index, edge_attr = data_0.edge_index, data_0.edge_attr
#     symb_model.conv.set_graph_attrs(edge_index, edge_attr)
#     for _ in range(T):
#         x = data_0.x[-1] + x + symb_model.conv(t=torch.tensor([], device = x.device), x=x)
#         pred.append(x)

#     return torch.stack(pred, dim=0)
    
    
from datasets.RealEpidemics import RealEpidemics

real_epid_data = RealEpidemics(
    root = './data_real_epid_covid_int',
    name = 'RealEpid',
    predict_deriv=False,
    history=1,
    horizon=44,
    scale=False
)
import json

with open('./data_real_epid_covid_int/RealEpid/countries_dict.json', 'r') as f:
    countries_dict = json.load(f)

In [2]:
class symb_wrapper(torch.nn.Module):
    def __init__(self, symb, is_self_interaction = True):
        super().__init__()
        self.symb = symb
        self.is_self_interaction = is_self_interaction
    
    def forward(self, x):
        if self.is_self_interaction:
            return self.symb(x_i=x[:, 0])
        else:
            return self.symb(x_i=x[:, 0], x_j=x[:, 1])
        

In [4]:
model_path = "./saved_models_optuna/model-real-epid-mpnn/real_epid_mpnn_7/0"

# import random

def build_symb_mpnn_to_opt():
    x_i, x_j = sp.symbols('x_i x_j')

    a = 3.7716758
    b = 1.9867662
    c = 1.2657967

    eps = 1e-3
    expr1 = sp.ln(sp.Max(sp.tan(x_i + c)**2 + 1, eps))
    expr2 = a * sp.ln(sp.Max(x_i + b, eps))

    g_symb = sympytorch.SymPyModule(expressions=[expr1])
    h_symb = sympytorch.SymPyModule(expressions=[expr2])
    
    g_symb = symb_wrapper(g_symb, is_self_interaction=False)
    h_symb = symb_wrapper(h_symb, is_self_interaction=True)

    symb_model = get_model(
        g = g_symb,
        h = h_symb,
        message_passing=False,
        include_time=False,
        integration_method='rk4',
        eval=False,
        all_t=True
    )

    symb_model = symb_model.train()   
    symb_model = symb_model.to('cuda:2')
    
    return symb_model

scaler = get_scaler(data = real_epid_data, tr_perc=0.8)

fit_param_per_country_gd(
    data=real_epid_data,
    build_symb_model=build_symb_mpnn_to_opt,
    countries_dict=countries_dict,
    model_path=model_path,
    epochs=3,
    lr=1e-3,
    patience=3,
    log=1,
    scaler=scaler,
    tr_perc=0.8,
    device="cuda:2"
)



Processing country Iceland
Epoch 0, train Loss: 0.0920, valid loss: 0.0206
Epoch 1, train Loss: 0.0015, valid loss: 0.0206
Epoch 2, train Loss: 0.0015, valid loss: 0.0206
Inferred coeffs for Iceland: [3.7718024e+00 9.9999997e-06 2.0018620e+00 9.9999997e-06 1.2658442e+00]

Processing country Canada
Epoch 0, train Loss: 0.0731, valid loss: 0.0243
Epoch 1, train Loss: 0.0016, valid loss: 0.0243
Epoch 2, train Loss: 0.0016, valid loss: 0.0243
Inferred coeffs for Canada: [3.7715943e+00 9.9999997e-06 1.9985843e+00 9.9999997e-06 1.2661270e+00]

Processing country Algeria
Epoch 0, train Loss: 0.0793, valid loss: 0.0115
Epoch 1, train Loss: 0.0014, valid loss: 0.0115
Epoch 2, train Loss: 0.0014, valid loss: 0.0115
Inferred coeffs for Algeria: [3.7715859e+00 9.9999997e-06 1.9997382e+00 9.9999997e-06 1.2658961e+00]

Processing country Burkina Faso
Epoch 0, train Loss: 0.0844, valid loss: 0.0158
Epoch 1, train Loss: 0.0012, valid loss: 0.0158
Epoch 2, train Loss: 0.0012, valid loss: 0.0158
Inferr

In [ ]:
scaler = get_scaler(data = real_epid_data, tr_perc=0.8)
x_i, x_j = sp.symbols('x_i x_j')
inf_coeff_covid = pd.read_csv("./saved_models_optuna/model-real-epid-mpnn/real_epid_mpnn_7/0/inferred_coeff.csv")


def build_symb_model_mpnn(country, inf_coeff_covid):
    
    coeffs = inf_coeff_covid[country]
    a, eps1, b, eps2, c, = coeffs.iloc[0], coeffs.iloc[1], coeffs.iloc[2], coeffs.iloc[3], coeffs.iloc[4] 

    g_symb = make_callable(sp.ln(sp.Max(sp.tan(x_i + c)**2 + 1, eps2)))
    h_symb = make_callable(a * sp.ln(sp.Max(x_i + b, eps1)))
    
    symb_model = get_model(
        g = g_symb,
        h = h_symb,
        message_passing=False,
        include_time=False,
        integration_method='dopri5'
    )
    
    return symb_model

y_true, y_pred, y_true_val, y_pred_val = eval_real_epid_int(
    data = real_epid_data,
    countries_dict=countries_dict,
    build_symb_model=build_symb_model_mpnn,
    scaler=scaler,
    use_euler=False,
    inferred_coeffs=inf_coeff_covid,
    tr_perc=0.8
)

In [13]:
print(f"Validation MAE: {mean_absolute_error(y_true_val.flatten(), y_pred_val.flatten())}")
print(f"Overall MAE: {mean_absolute_error(y_true.flatten(), y_pred.flatten())}")

Validation MAE: 1203.830078125
Overall MAE: 383.18951416015625
